In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 56.9 MB/s eta 0:00:00


In [ ]:
from pulp import *
from prettytable import *

## Conjuntos
  
$D$ = Dias de la semana, $d = [Lunes, Martes, ... , Sabado, Domingo]$.  
$H$ = Horas de funcionamiento, $h = [8, 9, ... , 20, 21]$.   

In [ ]:
# Conjuntos
dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
horas = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

## Parámetros

$o_{d,h}$ = Ordenes por entregar el día $d$ en la hora $h$, en ordenes.  
$i_{d,h}$ = Personal interno el día $d$ en la hora $h$, en personas.  
$pi$ = Productividad del personal interno, en $\frac{ordenes}{hora}$.  
$pe$ = Productividad del personal externo, en $\frac{ordenes}{hora}$.  
$ci$ = Costo del personal interno por hora, en $\frac{um}{hora}$.  
$ce$ = Costo de subcontratar personal externo, en $\frac{um}{turno}$.  
$tp$ = Valor promedio de una orden, en $um$.  
$toi$ = Tiempo de preparación de una orden por trabajador interno, en $hora$.  
$toe$ = Tiempo de preparación de una orden por trabajador externo, en $hora$.     



In [ ]:
# Parámetros

#Ordenes modelo 1 semana 35 tienda 104
demanda = {'Lunes'     : [0, 3, 2, 2, 2, 1, 2, 3, 3, 3, 3, 3, 0, 0],
           'Martes'    : [0, 3, 3, 2, 1, 4, 4, 2, 4, 4, 5, 5, 0, 0],
           'Miercoles' : [0, 4, 5, 5, 5, 7, 7, 6, 5, 7, 7, 11, 0, 0],
           'Jueves'    : [0, 5, 6, 5, 4, 5, 5, 5, 4, 5, 8, 8, 0, 0],
           'Viernes'   : [0, 3, 5, 5, 9, 6, 5, 6, 6, 7, 8, 9, 0, 0],
           'Sabado'    : [0, 5, 7, 6, 6, 7, 10, 8, 6, 6, 7, 8, 0, 0],
           'Domingo'   : [0, 4, 6, 5, 6, 10, 8, 7, 8, 7, 5, 9, 0, 0]
           }
#Datos semana 35 tienda 104
empleados_internos = {'Lunes'     : [2, 2, 2, 4, 5, 5, 5, 4, 3, 4, 3, 3, 0, 0],
                      'Martes'    : [2, 2, 2, 3, 4, 3, 4, 4, 3, 3, 2, 2, 0, 0],
                      'Miercoles' : [2, 2, 2, 4, 5, 4, 5, 4, 4, 4, 3, 3, 0, 0],
                      'Jueves'    : [1, 1, 1, 3, 4, 4, 4, 3, 3, 3, 3, 3, 0, 0],
                      'Viernes'   : [2, 2, 2, 3, 4, 3, 4, 4, 3, 3, 2, 2, 0, 0],
                      'Sabado'    : [3, 4, 4, 4, 5, 6, 5, 6, 4, 5, 4, 3, 0, 0],
                      'Domingo'   : [1, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 2, 0, 0]
                      }

productividad_internos = 1.9
productividad_externos = 1.4
costo_externo = 43330
costo_interno = 7612
ticket_promedio = 50000

t_orden_interno = 1/productividad_internos
t_orden_externo = 1/productividad_externos

## Variables de desición

$x\_am_{d}$ = Turnos de personal externo AM a subcontratar el dia $d$.  
$x\_pm_{d}$ = Turnos de personal externo PM a subcontratar el dia $d$.  
$w_{d,h}$ = Cantidad de ordenes adelantadas el dia $d$ en el slot $h$.  
$y\_i_{d,h}$ = Ordenes preparadas por el personal interno el dia $d$ en el slot $h$.  
$y\_e_{d,h}$ = Ordenes preparadas por el personal externo el dia $d$ en el slot $h$.  



In [ ]:
# Variables de decisión
x_am = LpVariable.dicts(name="Externos_AM",
                        indices=dias,
                        lowBound=0,
                        cat='Integer')

x_pm = LpVariable.dicts(name="Externos_PM",
                        indices=dias,
                        lowBound=0,
                        cat='Integer')

adelantado = LpVariable.dicts(name="Pedidos_Adelantados",
                              indices=[(dia, hora) for dia in dias for hora in horas[:-1]],
                              lowBound=0,
                              cat='Integer')

ordenes_internos = LpVariable.dicts("Trabajo_Internos",
                                    indices=[(dia, hora) for dia in dias for hora in horas],
                                    lowBound=0,
                                    cat='Integer')

ordenes_externos = LpVariable.dicts("Trabajo_Externos",
                                    indices=[(dia, hora) for dia in dias for hora in horas],
                                    lowBound=0,
                                    cat='Integer')

## Función objetivo

$min$ $z$ = $\frac{ce\ \sum_{d=Lunes}^{d=Domingo}{(x\_am_{d} + x\_pm_{d})} \ + \ ci \ \sum_{d=Lunes}^{d=Domingo}{\sum_{h=8}^{h=21}{(y\_i_{d,h} \ toi)}}  }{tp \ \ \sum_{d=Lunes}^{d=Domingo}{\sum_{h=8}^{h=21}{o_{d,h}}}}$

In [ ]:
# Función Objetivo
prob = LpProblem("Minimizar_Costo_sobre_Venta", LpMinimize)
prob += (costo_externo * lpSum(x_am[dia] + x_pm[dia] for dia in dias) + costo_interno * (lpSum(ordenes_internos[(dia, hora)]*t_orden_interno for hora in horas for dia in dias))) / (ticket_promedio * lpSum(demanda[dia][i] for dia in dias for i in range(len(horas)))), "Costo_sobre_venta"

## Restricciones

### Restricción para cubrir la demanda por slot


$i_{d,8} \ pi \ + \ x\_am_{d} \ pe ≥ o_{d,8} + \ w_{d,8} \ \ \ \ \ d \in D$

$ i_{d,h} \ pi \ + \ x\_am_{d} \ pe ≥ o_{d,h} + \ w_{d,h} \ - \ w_{d,h-1} \ \ \ \ \ d \in D, \forall \ h \in \{9,10\} $

$ i_{d,h} \ pi \ + \ x\_am_{d} \ pe \ + \ x\_pm_{d} \ pe ≥ o_{d,h} + \ w_{d,h} \ - \ w_{d,h-1} \ \ d \in D, \forall \ h \in \{11,17\} $

$ i_{d,h} \ pi \ + \ x\_pm_{d} \ pe ≥ o_{d,h} + \ w_{d,h} \ - \ w_{d,h-1} \ \ d \in D, \forall \ h \in \{18,19\} $

$ i_{d,20} \ pi ≥ o_{d,20} + \ w_{d,20} \ - \ w_{d,19} \ \ d \in D$

$ i_{d,21} \ pi ≥ o_{d,21} - \ w_{d,20} \ \ d \in D$





In [ ]:
# Restricciones de Demanda y Adelantamiento
for dia in dias:
    for i, hora in enumerate(horas):

        if hora == 8:
            prob += (empleados_internos[dia][i] * productividad_internos + x_am[dia] * productividad_externos) >= demanda[dia][i] + adelantado[(dia, hora)], f"Capacidad_slot_{dia}_{hora}h"

        elif 9 <= hora <= 10:
            prob += (empleados_internos[dia][i] * productividad_internos + x_am[dia] * productividad_externos) >= demanda[dia][i] + adelantado[(dia, hora)] - adelantado[(dia, horas[i-1])], f"Capacidad_slot_{dia}_{hora}h"

        elif 11 <= hora <= 17:
            prob += (empleados_internos[dia][i] * productividad_internos + x_am[dia] * productividad_externos + x_pm[dia] * productividad_externos) >= demanda[dia][i] + adelantado[(dia, hora)] - adelantado[(dia, horas[i-1])], f"Capacidad_slot_{dia}_{hora}h"

        elif 18 <= hora <= 19:
            prob += (empleados_internos[dia][i] * productividad_internos + x_pm[dia] * productividad_externos) >= demanda[dia][i] + adelantado[(dia, hora)] - adelantado[(dia, horas[i-1])], f"Capacidad_slot_{dia}_{hora}h"

        elif hora == 20:
            prob += (empleados_internos[dia][i] * productividad_internos) >= demanda[dia][i] + adelantado[(dia, hora)] - adelantado[(dia, horas[i-1])], f"Capacidad_slot_{dia}_{hora}h"

        else:
            prob += (empleados_internos[dia][i] * productividad_internos) >= demanda[dia][i] - adelantado[(dia, horas[i-1])], f"Capacidad_slot_{dia}_{hora}h"

### Restricción de capacidad de adelantamiento del siguiente slot

$ w_{d,h} ≤ o_{d,h+1} \ \ d \in D, \ h \in H $

In [ ]:
# Restricción de Adelantamiento condicional
for dia in dias:
    for i, hora in enumerate(horas[:-1]):

        prob += adelantado[(dia, hora)] <= demanda[dia][i+1], f"Ordenes_adelantadas_{dia}_slot-{hora}"

### Restricción de adelantamiento condicional del siguiente slot

$ w_{d,h} ≤ i_{d,h} pi \ + \ x\_am_ \ pe \ \ d \in D, \forall h \in \{8, 10\}$

$ w_{d,h} ≤ i_{d,h} pi \ + \ x\_am_ \ pe \ + \ x\_pm_ \ pe \ \ d \in D, \forall h \in \{11, 17\}$

$ w_{d,h} ≤ i_{d,h} pi \ + \ x\_pm_ \ pe \ \ d \in D, \forall h \in \{18, 19\}$

$ w_{d,h} ≤ i_{d,h} pi \ \ d \in D, \forall h \in \{20, 21\}$

In [ ]:

# Restricción de Adelantamiento condicional
for dia in dias:
    for i, hora in enumerate(horas[:-1]):

        if 8 <= hora <= 10:
            prob += adelantado[(dia, hora)] <= empleados_internos[dia][i] * productividad_internos + x_am[dia] * productividad_externos

        elif 11 <= hora <= 17:
            prob += adelantado[(dia, hora)] <= empleados_internos[dia][i] * productividad_internos + x_am[dia] * productividad_externos + x_pm[dia] * productividad_externos

        elif 18 <= hora <= 19:
            prob += adelantado[(dia, hora)] <= empleados_internos[dia][i] * productividad_internos + x_pm[dia] * productividad_externos

        else:
            prob += adelantado[(dia, hora)] <= empleados_internos[dia][i] * productividad_internos

### Restricción de ordenes a preparar por internos

$ y\_i_{d,h} ≥ i_{d,h} \ pi \ \ d \in D, \ h \in H$



In [ ]:
for dia in dias:
    for i, hora in enumerate(horas):

        prob += ordenes_internos[(dia, hora)] >= empleados_internos[dia][i] * productividad_internos, f"Capacidad_Internos_{dia}-slot{hora}"


### Restricción de capacidad de ordenes preparadas por externos

$ y\_e_{d,h} ≤ x\_am_{d} \ pe \ \ \ d \in D,  \ \ \ h ∈ \{8,10\} $

$ y\_e_{d,h} ≤ x\_am_{d} \ pe \ + \ x\_pm_{d} \ pe \ \ \ d \in D, \ \ \ h \in \{11,17\} $

$ y\_e_{d,h} ≤ x\_pm_{d} \ pe \ \ \ d \in D, \ \ \ h \in \{18,19\} $

$ y\_e_{d,h} ≤ 0 \ \ \ d \in D, \ \ \ h ∈ \{20,21\} $


In [ ]:
for dia in dias:
    for i, hora in enumerate(horas):

        if 8 <= hora <= 10:
            prob += ordenes_externos[(dia, hora)] <= x_am[dia] * productividad_externos, f"Capacidad_Externos_AM_{dia}-slot{hora}"

        elif 11 <= hora <= 17:
            prob += ordenes_externos[(dia, hora)] <= x_am[dia] * productividad_externos + x_pm[dia] * productividad_externos, f"Capacidad_Externos_PM_{dia}-slot{hora}"

        elif 18 <= hora <= 19:
            prob += ordenes_externos[(dia, hora)] <= x_pm[dia] * productividad_externos, f"Capacidad_Externos_PM_{dia}-slot{hora}"

        else:
            prob += ordenes_externos[(dia, hora)] <= 0


### Restricción de ordenes maximas a preparar por slot

$ y\_e_{d,8} \ + \ y\_i_{d,8} ≥ o_{d,8} \ + \ w_{d,8} \ \ d \in D$

$ y\_e_{d,h} \ + \ y\_i_{d,h} ≥ o_{d,h} \ + \ w_{d,h} \ - \ w_{d,h-1} \ \ d \in D, \forall \ h \in \{9,21\}$

$ y\_i_{d,20} ≥ o_{d,20} \ + \ w_{d,20} \ - \ w_{d,19} \ \ d \in D$

$ y\_i_{d,21} ≥ o_{d,21} \ - \ w_{d,20} \ \ d \in D$




In [ ]:
for dia in dias:
    for i, hora in enumerate(horas):

        if hora == 8:
            prob += ordenes_externos[(dia, hora)] + ordenes_internos[(dia, hora)]  >= demanda[dia][i] + adelantado[(dia, hora)]

        elif 9 <= hora <= 19:
            prob += ordenes_externos[(dia, hora)] + ordenes_internos[(dia, hora)]  >= demanda[dia][i] + adelantado[(dia, hora)] - adelantado[(dia, horas[i-1])]

        elif hora == 20:
            prob += ordenes_internos[(dia, hora)] >= demanda[dia][i] + adelantado[(dia, hora)] - adelantado[(dia, horas[i-1])]

        else:
            prob += ordenes_internos[(dia, hora)] >= demanda[dia][i] - adelantado[(dia, horas[i-1])]


## Solución

In [ ]:
# Resolver el problema
prob.solve()

print(f"Estado de la solución: {LpStatus[prob.status]}")

Estado de la solución: Optimal


In [ ]:
# Resultados
turnos = PrettyTable()
turnos.field_names = ["Dia", "Turnos AM", "Turnos PM"]
turnos.set_style(DOUBLE_BORDER)
turnos.align["Dia"] = "l"
turnos.align["Turnos AM"] = "r"
turnos.align["Turnos PM"] = "r"

for dia in dias:
    turnos.add_row([dia, x_am[dia].varValue, x_pm[dia].varValue])

print(f"Costo total de operar: {round(value(prob.objective)*100,2)}%")
print(turnos)

Costo total de operar: 11.62%
╔═══════════╦═══════════╦═══════════╗
║ Dia       ║ Turnos AM ║ Turnos PM ║
╠═══════════╬═══════════╬═══════════╣
║ Lunes     ║       0.0 ║       0.0 ║
║ Martes    ║       0.0 ║       0.0 ║
║ Miercoles ║       0.0 ║       1.0 ║
║ Jueves    ║       2.0 ║       0.0 ║
║ Viernes   ║       0.0 ║       1.0 ║
║ Sabado    ║       0.0 ║       0.0 ║
║ Domingo   ║       1.0 ║       1.0 ║
╚═══════════╩═══════════╩═══════════╝


In [ ]:
ordenes_i = PrettyTable()
ordenes_i.field_names = ["Dia", "Slot 8", "Slot 9","Slot 10","Slot 11","Slot 12","Slot 13","Slot 14","Slot 15","Slot 16","Slot 17","Slot 18","Slot 19","Slot 20","Slot 21"]
ordenes_i.set_style(DOUBLE_BORDER)
ordenes_i.align["Dia"] = "l"

for dia in dias:
    ordenes_i.add_row([dia,
                       ordenes_internos[(dia, 8)].varValue,
                       ordenes_internos[(dia, 9)].varValue,
                       ordenes_internos[(dia, 10)].varValue,
                       ordenes_internos[(dia, 11)].varValue,
                       ordenes_internos[(dia, 12)].varValue,
                       ordenes_internos[(dia, 13)].varValue,
                       ordenes_internos[(dia, 14)].varValue,
                       ordenes_internos[(dia, 15)].varValue,
                       ordenes_internos[(dia, 16)].varValue,
                       ordenes_internos[(dia, 17)].varValue,
                       ordenes_internos[(dia, 18)].varValue,
                       ordenes_internos[(dia, 19)].varValue,
                       ordenes_internos[(dia, 20)].varValue,
                       ordenes_internos[(dia, 21)].varValue])

print(ordenes_i)

╔═══════════╦════════╦════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╗
║ Dia       ║ Slot 8 ║ Slot 9 ║ Slot 10 ║ Slot 11 ║ Slot 12 ║ Slot 13 ║ Slot 14 ║ Slot 15 ║ Slot 16 ║ Slot 17 ║ Slot 18 ║ Slot 19 ║ Slot 20 ║ Slot 21 ║
╠═══════════╬════════╬════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╣
║ Lunes     ║  4.0   ║  4.0   ║   4.0   ║   8.0   ║   10.0  ║   10.0  ║   10.0  ║   8.0   ║   6.0   ║   8.0   ║   6.0   ║   6.0   ║   0.0   ║   0.0   ║
║ Martes    ║  4.0   ║  4.0   ║   4.0   ║   6.0   ║   8.0   ║   6.0   ║   8.0   ║   8.0   ║   6.0   ║   6.0   ║   4.0   ║   4.0   ║   0.0   ║   0.0   ║
║ Miercoles ║  4.0   ║  4.0   ║   4.0   ║   8.0   ║   10.0  ║   8.0   ║   10.0  ║   8.0   ║   8.0   ║   8.0   ║   6.0   ║   6.0   ║   0.0   ║   0.0   ║
║ Jueves    ║  2.0   ║  2.0   ║   2.0   ║   6.0   ║   8.0   ║   8.0   ║   8.0   ║   6.0 

In [ ]:
ordenes_e = PrettyTable()
ordenes_e.field_names = ["Dia", "Slot 8", "Slot 9","Slot 10","Slot 11","Slot 12","Slot 13","Slot 14","Slot 15","Slot 16","Slot 17","Slot 18","Slot 19","Slot 20","Slot 21"]
ordenes_e.set_style(DOUBLE_BORDER)
ordenes_e.align["Dia"] = "l"

for dia in dias:
    ordenes_e.add_row([dia,
                       round(ordenes_externos[(dia, 8)].varValue,2),
                       round(ordenes_externos[(dia, 9)].varValue,2),
                       round(ordenes_externos[(dia, 10)].varValue,2),
                       round(ordenes_externos[(dia, 11)].varValue,2),
                       round(ordenes_externos[(dia, 12)].varValue,2),
                       round(ordenes_externos[(dia, 13)].varValue,2),
                       round(ordenes_externos[(dia, 14)].varValue,2),
                       round(ordenes_externos[(dia, 15)].varValue,2),
                       round(ordenes_externos[(dia, 16)].varValue,2),
                       round(ordenes_externos[(dia, 17)].varValue,2),
                       round(ordenes_externos[(dia, 18)].varValue,2),
                       round(ordenes_externos[(dia, 19)].varValue,2),
                       round(ordenes_externos[(dia, 20)].varValue,2),
                       round(ordenes_externos[(dia, 21)].varValue,2)])

print(ordenes_e)

╔═══════════╦════════╦════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╦═════════╗
║ Dia       ║ Slot 8 ║ Slot 9 ║ Slot 10 ║ Slot 11 ║ Slot 12 ║ Slot 13 ║ Slot 14 ║ Slot 15 ║ Slot 16 ║ Slot 17 ║ Slot 18 ║ Slot 19 ║ Slot 20 ║ Slot 21 ║
╠═══════════╬════════╬════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╬═════════╣
║ Lunes     ║  0.0   ║  0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║
║ Martes    ║  0.0   ║  0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║
║ Miercoles ║  0.0   ║  0.0   ║   0.0   ║   1.0   ║   1.0   ║   1.0   ║   1.0   ║   1.0   ║   1.0   ║   1.0   ║   0.0   ║   0.0   ║   0.0   ║   0.0   ║
║ Jueves    ║  2.0   ║  2.0   ║   2.0   ║   2.0   ║   2.0   ║   2.0   ║   2.0   ║   2.0 

In [ ]:
# Mostrar el modelo completo
#print("\nModelo completo:")
#print(prob)

#for v in prob.variables():
#    print(f"{v.name} = {v.varValue}")